In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import product
from sklearn.preprocessing import MinMaxScaler

from simulation.vizualization import Visualizer

In [2]:
warnings.filterwarnings('ignore')

In [3]:
def load_data(path_to_dir: str) -> pd.DataFrame:
    files = [x for x in os.listdir(path_to_dir) if '.pq' in x]
    df_list = []
    
    for file in files:
        sub = pd.read_parquet(os.path.join(path_to_dir, file))
        # sub['city'] = file.replace('simulation_results_', '').split('_')[0]
        sub['driver_frac'] = round(sub['driver_frac'], 3)
        # horizon = file.replace('simulation_results_', '').split('_')[1]
        
        # if 'min' in horizon:
        #     sub['horizon'] = horizon
        # else:
        #     sub['horizon'] = '3min'
            
        df_list.append(sub)
        
    return pd.concat(df_list, ignore_index=True)

In [4]:
data = load_data('sim_res')

### Visualization

In [5]:
viz = Visualizer()

In [ ]:
viz.draw_heatmap(
    data,
    #filter_list=[
    #    'greedy_sequential_matching',
    #    'composite_sequential_matching',
    #    'k_hungarian_capacity_exhaust',
    #    'k_hungarian_matching'
    #],
    save=False,
    optimal=False
)

In [6]:
viz.plot_counts(data, save=False)

interactive(children=(Dropdown(description='city', options=('riyadh', 'jeddah'), value='riyadh'), Dropdown(des…

In [7]:
viz.plot_surface(data, save=False)

interactive(children=(Dropdown(description='city', options=('riyadh', 'jeddah'), value='riyadh'), Dropdown(des…

In [ ]:
viz.plot_line(data)

In [ ]:
viz.plot_static(data)

### Additional plots

In [ ]:
data.head()

In [ ]:
df = (
    data[
        (data.m <= 10) &
        (data.m > 1) &
        (data.capacity <= 10) &
        (data.capacity > 1) &
        (data.driver_frac < 1.0) &
        (data.driver_frac > 0.05)
    ]
).copy()

df['SR'] = 1 - df['num_clients_with_no_handshake_options'] / df['num_clients']

df_list = []

for driver_frac in df.driver_frac.unique().tolist():
    sub = df[df.driver_frac == driver_frac].copy()
    
    scaler = MinMaxScaler()
    sub['SR'] = scaler.fit_transform(sub['SR'].values.reshape(-1, 1))
    df_list.append(sub)
    
df = pd.concat(df_list).sort_index(axis=0)

In [ ]:
df = df[['matching_fn', 'capacity', 'm', 'n_possible', 'SR']]
df['matching_fn'] = (
    df['matching_fn'].str.replace('k_hungarian_capacity_exhaust', 'k_hungarian_one_sided')
    .str.replace('k_hungarian_matching', 'fixed_k_hungarian')
    .str.replace('k_hungarian_m_capacity_exhaust', 'k_hungarian_two_sided')
)

In [ ]:
df.head()

In [ ]:
df_grouped = df.groupby(['matching_fn', 'capacity', 'm', 'n_possible'], as_index=False).mean()

In [ ]:
df_grouped.head()

In [ ]:
def heatmap(data: pd.DataFrame, L: int) -> None:
    heat = data[data.n_possible == L].copy()
    heat['param'] = heat.apply(lambda row: str(dict(capacity=row['capacity'], M=row['m'])), axis=1)
    heat = heat.sort_values(['capacity', 'm']).drop(['capacity', 'm'], axis=1)
    df_heat = pd.pivot_table(heat, 'SR', 'param', 'matching_fn', sort=False)
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 20))  # int(np.ceil(2.23*(10-(L-1))))
    heatmap = sns.heatmap(df_heat, annot=True, fmt='.3f', ax=ax, cbar=False, cmap="Spectral")
    heatmap.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
    plt.xticks(rotation=30)
    plt.xlabel('')
    # plt.savefig(f'rfc_pics/heatmap_avg_l_{L}.png', bbox_inches='tight', pad_inches=0.5)
    plt.show()

In [ ]:
heatmap(df_grouped, 5)

In [ ]:
df_grouped_L = df.groupby(['matching_fn', 'n_possible'], as_index=False).mean()

In [ ]:
sns.lineplot(df_grouped_L, x='n_possible', y='SR', hue='matching_fn')
plt.xticks(ticks=[1, 2, 3, 4, 5])
plt.xlabel('L')
plt.ylabel('Normalized Success Rate')
plt.savefig('rfc_pics/L_comp.png', bbox_inches='tight', pad_inches=0.5)
plt.show()

In [ ]:
len(data)